In [1]:
%cd ..

/media/axeh/DATA/code_stuff/python_stuff/projects/reaction_diffusion_tutorial


In [2]:
import numpy as np
from scipy.ndimage import convolve
from tqdm import tqdm_notebook as tqdm

from bokeh.plotting import figure
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import ColumnDataSource
from  ipywidgets import interact
output_notebook()

from recaps.utils import euler_forward, runge_kutta2, convert2img

Loading BokehJS ...

A verrry old paper by [Kondo and Asal, 1995](http://www.fbs.osaka-u.ac.jp/labs/skondo/paper/kondo%20Nature%201995.pdf) describes how hypothetical reaction-diffusion model can help to gain some insight on the possible mechanism behind the formation of stripe patterns of marine angelfish _Pomacanthus_. Here's some background info about the fish, as given in the paper:

> _The marine angelfish, Pomacanthus, has stripe patterns, which are not fixed on their skin. Unlike mammal skin patterns, which simply enlarge proportionally during their body growth, the stripes of Pomacanthus maintain the spaces betrween the lines by the continuous rearrangements of the patterns._

> <img src="..\images\angelfish.png"></img>
> <body><center><font size=2>The stripe patterns of adult <i>Pomacanthus imperator</i> (<a href="http://www.fbs.osaka-u.ac.jp/labs/skondo/paper/kondo%20Nature%201995.pdf">img</a>)</font></center></body>

Fascinating! And, as it turned out, these patterns can be mimicked _in silico_ with a reaction-diffusion system of two hypothetical molecules: one acting as activator $x_A$ and the other - as inhibitor $x_I$. Obviously, we're going to replicate it. 

The paper suggests the following system of equations to replicate the angelfish patterns:

$$ \begin{cases} \frac{dx_A}{dt} = D_A\nabla^2 x_A + c_1 x_A + c_2 x_I + c_3 - g_A x_A \\
                 \frac{dx_I}{dt} = D_I\nabla^2 x_I + c_4 x_A + c_5 - g_I x_I \end{cases} $$

where  

<table>
    <tr><td> <b>Symbol</b>  </td> <td> <b>Value</b> </td> <td> <b>Interpretation</b> </td> </tr>
<tr><td> $D_A$ </td> <td> 0.007 </td> <td> Diffusion constant of activator </td> </tr>
<tr><td> $D_I$ </td> <td> 0.1   </td> <td> Diffusion constant of inhibitor </td> </tr>
<tr><td> $c_1$ </td> <td> 0.08  </td> <td> Synthesis rate of activator </td> </tr>
<tr><td> $c_2$ </td> <td>-0.08  </td> <td> Inhibition rate </td> </tr>
<tr><td> $c_3$ </td> <td> 0.05  </td> <td> Activator input term </td> </tr>
<tr><td> $c_4$ </td> <td> 0.1   </td> <td> Activation rate </td> </tr>
<tr><td> $c_5$ </td> <td>-0.05  </td> <td> Inhibitor output term </td> </tr>
<tr><td> $g_A$ </td> <td> 0.03  </td> <td> Decay rate of activator </td> </tr>
<tr><td> $g_I$ </td> <td> 0.06  </td> <td> Decay rate of inhibitor</td> </tr>
</table>

Additionally, authors suggest to bound the net synthesis rate of activator and inhibitor to avoid unrealistic situations:

$$ \begin{cases} 0 < c_1 x_A + c_2 x_I + c_3 < 0.18 \\
                 0 < c_4 x_A + c_5 < 0.5 \end{cases} $$
                 
> <font size=2>This is not particularly elegant, but it's not very crucial for us, since the purpose of this notebook is just to test the code and glance over various patterns that can be formed in the reaction-diffusion system. </font>

To implement this system, we can just copy-paste the code, that we already wrote, and modify the reaction term.

In [3]:
def fish(_, x, p):
    x = x.reshape(p['size'])   # resize flat 1D array back into (height,width,#channels)-array
    dxdt = np.zeros(p['size']) # preallocate the accumulation terms    
    diffusion = np.zeros(p['size']) # preallocate the diffusion term
    reaction  = np.zeros(p['size']) # preallocate the reaction term
    
    # --- get diffusion term ---
    laplacian = np.array([[0,  1, 0], 
                          [1, -4, 1], 
                          [0,  1, 0]])    
    for k in range(p['size'][2]):   
        diffusion[:,:,k] = p['D'][k]/p['h']**2 * convolve(x[:,:,k], laplacian, mode="nearest")
        
    # --- get reaction term ---
    x_a, x_i = x[:,:,0], x[:,:,1]
    reaction[:,:,0] = np.clip(p['c1']*x_a + p['c2']*x_i + p['c3'], 0, 0.18) - p['ga']*x_a
    reaction[:,:,1] = np.clip(p['c4']*x_a + p['c5'], 0, 0.5) - p['gi']*x_i    
        
    # --- get total accumulation term ---
    dxdt = diffusion + reaction
        
    return dxdt.ravel()


# parameters: system 
p = {'ga': 0.03,       # decay rate of activator
     'gi': 0.06,       # decay rate of inhibitor  
     'c1': 0.08,       # synthesis rate of activator
     'c2':-0.08,       # inhibition rate
     'c3': 0.05,       # constant activator input term
     'c4': 0.1,        # activation rate
     'c5':-0.15,       # constant inhibitor output term
     'D' :[0.007, 0.1]}# diffusion rates 

# parameters: spatial grid
resolution = [75,75]          # grid dimensions in pixels
p['size']  = (*resolution, 2) # (height, width, #states)
p['h']     = 0.5              # pixel size in physical units


# time-related
t0, tf, dt = 0, 3000, 0.25
t_span = np.arange(t0, tf+dt, dt)

# initial condition
x0 = np.random.rand(*p['size']).ravel()

# run the simulation: use euler_forward or runge_kutta2 to find the RGB "concentrations" numerically
img_development = euler_forward(fish, x0, t_span, p)


In [4]:
# plot stuff!
img_data = ColumnDataSource({'img': [np.flipud(convert2img(img_development[:,0], p['size']))]})

pb = figure(x_range=(0,p['size'][0]), y_range=(0,p['size'][1]),
            plot_width=p['size'][0]*4, plot_height=p['size'][1]*4)
r = pb.image_rgba(image='img', source=img_data, x=0, y=0, dw=p['size'][0], dh=p['size'][1])
show(pb, notebook_handle=True)  

def update(t=0):
    i = int(t/dt)
    # update data_source
    #(adjust the contrst to make patterns more visible)
    r.data_source.data['img'] = [np.flipud(convert2img(img_development[:,i], p['size'], 
                                                       steepness=1.5, midpoint=2))]
    push_notebook()
interact(update, t=(t0,tf,dt))

interactive(children=(FloatSlider(value=0.0, description='t', max=3000.0, step=0.25), Output()), _dom_classes=…

<function __main__.update(t=0)>

Notice how initially the system is almost homogeneous - there are no spatial patterns. This homogeneous system appears to oscillate wildly at the early stages. However, as time passes these temporal bulk oscillations gradually decay and static spatial patterns become more and more distinctive. The exact spatial pattern would be different each time the model is rerun with different initial conditions. However, as long as the parameter values remain the same, all the generated patterns will have the same signature.